In [3]:
import json
import requests
from pprint import pprint
import numpy as np
import pandas as pd
import re
import pickle
from sklearn.metrics import classification_report,jaccard_similarity_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report,jaccard_similarity_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [4]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE documents
             (tekst text, ids text, agenda int, bestuurEnBeleid int, veiligheid int, onderwijs int, 
             economie int, verkeer int, sociaal int, wonen int, gezondheid int, cultuur int)''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [14]:
con = sqlite3.connect('example.db')
cursor = con.cursor()
cursor.execute("SELECT cultuur FROM documents")
print(cursor.fetchall())
conn.close()

[(0.9000794368264913,), (0.8248939840755267,), (0.9726710095736524,), (0.9866998702550696,), (0.9000794368264913,), (0.8248939840755267,), (0.9726710095736524,), (0.9866998702550696,), (0.9000794368264913,), (0.8248939840755267,), (0.9726710095736524,), (0.9866998702550696,), (0.9994612705495116,), (0.9970773459172892,), (0.9774787250345098,), (0.8265334966279044,), (0.8838076067145738,), (0.8754124546241716,)]


In [5]:
def classify_new_documents(listOfDocs):
    classificationModel, tfidfModel = load_models()
    listOfDocs, listOfDocsMatrix = prepare(listOfDocs, tfidfModel)
    #for i in np.where(classificationModel[3].coef_[0] > 5)[0]:
    #    print(tfidfModel.get_feature_names()[i])
    listOfDocs = classify(listOfDocs, listOfDocsMatrix,classificationModel)
    add_documents(listOfDocs)
    return listOfDocs
    
    
def use_feedback(listOfDocIDs, listOfFeedback):
    
    return

In [6]:
beleidsWoorden = ['bestuur en beleid','facilitering','bevolkingsregister','straatnaamgeving','kadaster','kadastrale','stelposten','begrotingsruimte','integratieuitkeringen','decentralisatieuitkeringen','accountantscontrole','ombudsfunctie','rekenkamer','reiskosten','loonkosten', 'paspoorten', 'rijbewijzen', 'verkiezingen', 'documentaire informatievoorziening ', 'inkoop', 'ozb', 'onroerend zaakbelasting ', 'parkeerbelasting', 'hondenbelasting ', 'precariobeslasting ', 'reclamebelasting ', 'vennootschapsbelasting', 'algemene uitkering ', 'mutaties reserves', 'dividend nutsbedrijven']
veiligheidWoorden = ['veiligheid', 'bibob', 'criminaliteit','criminelen','inbraken','explosieven','radicalisering','doodschouw','buitengewoon opsporingsambtenaar','crisisbeheersing', 'brandweer', 'brandbestrijding', 'rampenbestrijding', 'halt', 'apv', 'boa', 'georganiseerde criminaliteit', 'wet wapen en munitie', 'antidiscriminatiebeleid', 'dierenbescherming ', 'politie', 'leges drank en horeca', 'openbare orde']
onderwijsWoorden = ['onderwijs', 'onderwijshuisvesting', 'school', 'scholen', 'basisschool', 'bewegingsonderwijs', 'leraren', 'vandalismebestrijding', 'schoolgebouwen', 'onderwijsbeleid', 'leerlingzaken', 'leerkrachten', 'volwasseneneducatie', 'peuterspeelzaal', 'peuterspeelzalen', 'leerlingbegeleiding', 'leerlingzorg', 'leerlingenvervoer', 'schooldeelname', 'leerplicht', 'schoolverlaten', 'basisonderwijs']
economieWoorden = ['economische ontwikkeling','economie','economische','winkeliers','ondernemers', 'toerisme', 'landbouw','veeteelt','visserij','promotie', 'toeristen', 'beurzen', 'beurs', 'jaarmarkten', 'forensenbelasting', 'toeristenbelasting', 'vermakelijkhedenretributies', 'bedrijvenloket', 'ondernemersloket', 'startende ondernemers', 'straathandel', 'markten', 'biz-bijdrage', 'marktgelden', 'grondexploitatie bedrijventerreinen', 'winkelgebieden', 'bedrijfslocaties']
verkeerWoorden = ['verkeer', 'verkeersbeleid', 'verkeersmaatregelen', 'verkeersveiligheid', 'wegen', 'pleinen', 'fietspad', 'voetpad', 'straten', 'civieltechnische kunstwerken', 'straatverlichting', 'gladheidbestrijding', 'sneeuwruimen', 'strooien', 'straatreiniging', 'zwerfafval', 'parkeerbeleid', 'parkeermeters', 'parkeervoorzieningen ', 'jachthaven ', 'bruggelden', 'passantenhaven', 'liggelden', 'havengelden', 'baggerwerkzaamheden ', 'zeehavens', 'binnenhavens ', 'doorgaande waterwegen', 'waterkering ', 'afwatering', 'openbaar vervoer', 'bus', 'tram', 'metro', 'veerdiensten', 'taxivervoer', 'busstation', 'metrostation', 'multimodaal knooppunt']
cultuurWoorden = ['sport en cultuur', 'sportbeleid', 'topsport', 'sportbeoefening', 'recreatie', 'sport', 'cultuur', 'recreatieve', 'sportaccommodaties','sportverenigingen', 'sportvelden', 'zwembad', 'schaatshal', 'trapveldje', 'voetbalveld', 'voetbal', 'hockey', 'volleybal', 'handbal', 'muziek', 'dans', 'toneel', 'kunst', 'cultuurparticipatie', 'cultuurpresentatie', 'cultuurproductie', 'cultuuruitingen', 'kunstenaars', 'kunstwerken', 'cultuureducatie', 'herdenking', 'musea', 'museum', 'expositie', 'archeologie','archeologisch', 'heemkunde', 'historische archieven', 'bibliotheken', 'bibliotheek','stadsgezicht','monumenten','monumentenwet','lokale pers', 'lokale omroep', 'lokale informatievoorziening', 'natuurbescherming', 'onderhoud van bos', 'vijvers', 'vijver', 'openbaar groen', 'betuining', 'hobbyclub', 'volkstuinvereniging', 'speelvoorziening', 'recreatievoorziening']
gezondheidsWoorden = ['volksgezondheid', 'milieu', 'gezondheidssituatie', 'gezondheid', 'gezondheidsbevordering', 'infectieziekten', 'vaccinaties', 'vaccinatie','prenatale', 'psychosociale hulp', 'centrum jeugd en gezin', 'ziekenvervoer', 'ambulance', 'riolering', 'oppervlaktewater', 'rioolwaterzuivering', 'rioolheffing', 'grondwater', 'afvalwater', 'hemelwater', 'afval', 'vuilophaal', 'reinigingsrechten', 'vuilstort', 'afvalstoffenheffing', 'milieubeheer', 'geluidhinder', 'bodemsanering', 'rud', 'regionale uitvoeringsdiensten', 'ongediertebestrijding', 'recycling', 'begraafplaatsen', 'begraafplaats','crematoria', 'grafrechten', 'luchtkwaliteit', 'milieubeleid', 'milieukwaliteit']
sociaalWoorden = ['sociaal domein', 'burgerparticipatie', 'amw', 'algemeen maatschappelijk werk', 'burgerinitiatieven', 'eenzaamheidsbestrijding', 'collectief aanvullend vervoer', 'kinderopvang', 'noodopvang vluchtelingen', 'vreemdelingen', 'wijkteams', 'inkomensregelingen', 'ioaw', 'ioaz', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen', 'bijstandverlening', 'inkomensregeling', 'participatiewet', 'loonkostensubsidies', 'bijstandsverlening', 'bijstand', 'schuldhulpverlening', 'beschut werken', 'work first', 'proefplaatsen', 'participatieplaatsen', 'vrijwilligerswerk', 'detacheringsbanen', 'erkenning van verworven competenties', 'evc', 'loonkostensubsidie', 'jobcoach', 'werkplekaanpassingen', 'werkvoorzieningen', 'doventolk', 'inkomensvrijlating', 'stimuleringspremies', 'onkostenvergoedingen', 'no-riskpolis', 'inburgering', 'cursus nederlands', 'loonwaardebepaling', 'besluit bijstandsverlening zelfstandigen', 'bbz', 'maatwerkdienstverlening', 'maatwerkvoorzieningen', 'wmo', 'woningaanpassingen', 'gehandicaptenparkeerkaart ', 'eigen bijdrage', 'mantelzorg', 'pgb', 'jeugd-ggz', 'jeugdzorg', 'gescaleerde zorg', 'beschermdwonenvoorzieningen', 'opvangvoorzieningen', 'vrouwenopvang', 'veilig thuis', 'aanpak huiselijk geweld', 'beschermd wonen', 'kinderbeschermingsmaatregelen', 'jeugdreclassering ', 'ouderbijdragen']
wonenWoorden = ['volkshuisvesting en ruimtelijke ordening', 'ruimtelijke ordening', 'bgt', 'basisregistratie grootschalige topografie', 'bestemmingsplan', 'bestemmingsplannen', 'grondbeleid', 'cai', 'breedband', 'glasvezel', 'grondexploitatie', 'grondverwerving', 'bouwrijpe gronden', 'bouwgrondcomplexen', 'gebiedsontwikkeling', 'woningvoorraad', 'huisvestingsvoorziening', 'basisregistratie adressen en gebouwen', 'bouwtoezicht', 'bag', 'woningbouw', 'woningverbetering', 'woonruimteverdeling', 'woningsplitsingsvergunning', 'woonvergunning', 'stedelijke vernieuwing', 'woningvoorraad', 'woonomgeving']
lijsten = [beleidsWoorden,veiligheidWoorden,onderwijsWoorden,economieWoorden,verkeerWoorden,sociaalWoorden,wonenWoorden,gezondheidsWoorden,cultuurWoorden]

In [59]:
def load_models():
    transformer = TfidfTransformer()
    tfidfModel = pickle.load(open("Models\\tfidf.pickle", "rb"))
    classificationModels = []
    for lijst in lijsten:
        classificationModels.append(pickle.load(open("Models\\"+lijst[0]+ ".pickle", "rb")))
    return classificationModels, tfidfModel

def preprocess(text):
    text = text.lower()
    text = re.sub("<\w*>", '', text)
    text = re.sub("<\w*\s\/>", '', text)
    text = re.sub("^https?:\/\/.*[\r\n]*", '', text)
    text = re.sub('\\\\x\S.', '', text)
    text = re.sub('[^a-z\s]', '', text)
    text = re.sub("\s+", " ", text)
    return text

def prepare(listOfDocs, tfidfModel):
    listOfDocs["text"] = listOfDocs["text"].apply(preprocess)
    listOfDocsMatrix = tfidfModel.transform(listOfDocs["text"])
    return listOfDocs, listOfDocsMatrix

def classify(listOfDocs, listOfDocsMatrix, classificationModels):
    listOfDocs = classifyAgenda(listOfDocs)
    for clf, lijst in zip(classificationModels, lijsten):
        listOfDocs[lijst[0]] = clf.predict_proba(listOfDocsMatrix)[:,0]
        regex = r"\b(" 
        for word in lijst:
            regex += word + "|" 
        regex = regex[:-1]
        regex += r")\b"
        extra = (listOfDocs['text'].str.count(regex) / listOfDocs["text"].str.split().apply(len)) * 5
        listOfDocs[lijst[0]] -= extra
    return listOfDocs

def classifyAgenda(listOfDocs):
    listOfDocs["agenda"] = 0
    listOfDocs["agenda"] = listOfDocs["text"].str.slice(0,25).str.contains("agenda")*(listOfDocs["text"].str.slice(0,30).str.contains("agendapunt|raadsvoorstel")==False)
    return listOfDocs

def add_documents(listOfDocs):
    listOfDocs.to_sql('documents', index=False, con=engine, if_exists='replace')
    purchases = engine.execute("SELECT * FROM documents").fetchall()
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.executemany('INSERT INTO documents VALUES (?,?,?,?,?,?,?,?,?,?,?,?)', purchases)
    conn.commit()    
    return

In [37]:
r = requests.get("http://api.openraadsinformatie.nl/v0/zwolle/events/7bd357c38299b966b59b25c6f3d50a2ebfd5751e")
data = r.json()
listOfData = [point["description"] for point in data["sources"]]

listOfID = [1,2,3,4,5,6,7,8,9,10,11]
listOfData = listOfData[:10]
listOfData.append("agenda")
d = {"text": listOfData[:11], "ids":listOfID[:11]}
df = pd.DataFrame(data=d)
print(classify_new_documents(df))

    ids                                               text  agenda  \
0     1  coperatie agrobosbouw nl aan het gemeentebestu...   False   
1     2  from to subject date attachments frank menger ...   False   
2     3  bijlage bij de voordracht van november documen...   False   
3     4  v o o r d r a c h t november documentnummer vv...   False   
4     5  geachte leden van de gemeenteraad graag vragen...   False   
5     6  benchmark duurzaam inkopen van vervoer een ond...   False   
6     7  brief bewoners stockholmstraat zwolle november...   False   
7     8  from to cc subject date anker ed frank klomp h...   False   
8     9  aan het college van burgemeester en wethouders...   False   
9    10  ui rt ro o ccdsgnfls j i luttenbergstraat post...   False   
10   11                                             agenda    True   

    bestuur en beleid  veiligheid  onderwijs  economische ontwikkeling  \
0            0.956038    0.995113   0.993016                  0.781758   
1          

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [41]:
print(preprocess(data["sources"][6]["description"]).split()[:100])


['ui', 'rt', 'ro', 'o', 'ccdsgnfls', 'j', 'i', 'luttenbergstraat', 'postbus', 'ontvangen', 'i', 'ro', 'gb', 'zwolle', 'vd', 'telefoon', 'fax', 'overijsselnl', 'nov', 'gemeente', 'zwolle', 'tav', 'de', 'griffier', 'de', 'heer', 'a', 'ten', 'have', 'postbus', 'ga', 'zwolle', 'postbusoverijsselnl', 'registratuur', 'kvk', 'iban', 'nlrabo', 'inlichtingen', 'bij', 'b', 'post', 'mw', 'smm', 'hoogeboomvan', 'der', 'valk', 'telefoon', 'smmhoogeboomoverljsselnl', 'onderwerpsamen', 'voor', 'goed', 'bestuur', 'kwaliteit', 'van', 'het', 'openbaar', 'bestuur', 'in', 'overijssel', 'najaar', 'datum', 'bijlagen', 'i', 'brief', 'samen', 'voor', 'goed', 'bestuur', 'ii', 'activiteitenschema', 'samen', 'voor', 'goed', 'bestuur', 'pagina', 'hierbij', 'ontvangt', 'u', 'in', 'afschrift', 'de', 'brief', 'samen', 'voor', 'goed', 'bestuur', 'en', 'bijbehorend', 'activiteitenschema', 'die', 'wij', 'naar', 'de', 'besturen', 'van', 'de', 'overijsselse', 'gemeenten', 'regios', 'en']


In [81]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

listOfData = []
for piece in splitter(200, preprocess(data["sources"][1]["description"] )) :
    listOfData.append(piece)

print(len(listOfData))
d = {"text": listOfData[:2], "ids":listOfID[:2]}
df = pd.DataFrame(data=d)
print(classify_new_documents(df))

2
   ids                                               text  agenda  \
0    1  from to subject date attachments frank menger ...   False   
1    2  naar buiten te treden in de media dat wunderli...   False   

   bestuur en beleid  veiligheid  onderwijs  economische ontwikkeling  \
0           0.990765    0.991629   0.987678                  0.904639   
1           0.994366    0.988411   0.942694                  0.815518   

    verkeer  sociaal domein  volkshuisvesting en ruimtelijke ordening  \
0  0.945511        0.989885                                  0.916642   
1  0.968402        0.976398                                  0.967854   

   volksgezondheid  sport en cultuur  
0         0.988593          0.817820  
1         0.987471          0.875236  


C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [82]:
print(listOfData[0])

from to subject date attachments frank menger statengriffiefryslannl drents parlement griffieleeuwardennl griffierassennl raadsgriffie statengriffieoverijsselnl griffiernoordoostpoldernl internationale snelle treinverbindingen prorail en het project wunderline bottleneck groningen bremen vrijdag november voordracht ps planuitwerkingsbesluit deel project wunderlinepdf bijlage voortgang project wunderline juli november pdf voordracht ps planuitwerkingsbesluit deel project wunderlinepdf pxicenetworkpng geachte leden van de raden en staten de in tekst bijgevoegde onderliggende mail stuur ik naar u met een reden het grootste gat in het internationaal voor nederland bevindt zich in het noordoostelijk deel van het land waarbij de omleiding van de trein naar berlijn via zwolle en het project wunderline elkaar niet bijten tussen beide hoofdverbindingen zit een permanent geografisch gat van gemiddeld kilometer beide bedienen dan ook iets andere geografische markten dit vanwege de studies die in 